#### Deep Learning

# Keras Guide
#### Francisco Maiocchi

***
### Introducción
Keras es la API de alto nivel para crear y entrenar modelos de deep learning. Es usado para prototipado rápido, investigación avanzada, producción y cuenta con tres ventajas claves:  

+ __User friendly:__ Keras es simple y está optimizado para los usos más comunes. Brinda información clara y entendible sobre los errores.  
+ __Modular:__ los modelo de Keras se construyen conectando distintos bloques.
+ __Facil de escalar:__ se pueden crear nuevos bloques, capas, loss functions para probar ideas e investigar. 

Este guía está basada en la que se encuentra en la web oficial de TensorFlow en https://www.tensorflow.org/guide/keras?hl=es

***
### Imports
tf.keras es la implementación de TensorFlow de la Keras API specification. Es una API de alto nivel para construir y entrenar modelos que incluye soporte para funciones especificas de TensorFlow, como eager execution, tf.data pipelines y estimadores. tf.keras hace que TensorFlow sea mucho más facil de usar sin sacrificar flexibilidad ni performance.

In [1]:
# Importo TensorFlow como tf
import tensorflow as tf
# Importo keras
from tensorflow import keras

# Librerias auxiliares
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)
print(tf.keras.__version__)

1.12.0
2.1.6-tf


tf.keras puede correr cualquier código compatible con Keras pero hay que tener en cuenta que:  
+ La versión de tf.keras del último TensorFlow puede no ser la misma que la versión de PyPI. Checkear tf.keras.version.  
+ Cuando se guardan los pesos de un modelo, por defecto tf.keras lo hace en su formato checkpoint. Si se quiere usar HDF5 se debe pasar save_format = 'h5' a la función. 

***
### Construcción de un modelo simple

#### Modelo secuencial
En keras, para construir un modelo se deben encadenar capas o layers. Generalmente, un modelo es un grafo de capas. El tipo de modelo más comun es una pila de capas que se contruye mediante el modelo tf.keras.Sequential.  

Para construir una simple red fully-connected (perceptron multi capa): 

In [3]:
model = tf.keras.Sequential()
# Agrega una capa densamente conectada o fully-connected de 64 neuronas con activación 
model.add(keras.layers.Dense(64, activation='relu'))
# Se agrega otra capa
model.add(keras.layers.Dense(64, activation='relu'))
# Se agrega una capa con activación softmax de 10 neuronas
model.add(keras.layers.Dense(10, activation='softmax'))

#### Configurando el modelo
Existen un monton de capas disponibles en tf.keras.layers. La mayoría comparten estos parametros:  

+ __activation__: función activación de la capa. Este parametro se especifica con el nombre de una función ya implementada o como un callable object. Por defecto, ninguna activación es aplicada.  
+ __kernel_initializer__ y __bias_initializer__: el esquema de inicialización que crea los pesos de la capa. Este parametro es un nombre o un callable object. Por defecto se utiliza el inicializador __"Glorot uniform"__. 
+ __kernel_regularizer__ y __bias_regularizer__: el esquema de regularización que se aplica a los pesos de la capa, como por ejemplo L1 y L2 regularization. Estas técnicas se utilizan para prevenir el overfitting o sobreentrenamiento. Por defecto, no se utiliza ninguna regularización.  

Como ejemplo, vamos a instanciar una tf.keras.layers.Dense y ver sus argumentos de construcción.

In [4]:
# Crea una capa sigmoid
keras.layers.Dense(64, activation='sigmoid')
# O:
keras.layers.Dense(64, activation=tf.sigmoid)

# Una capa lineal con L1 regularization de factor 0.01 aplicado al kernel
keras.layers.Dense(64, kernel_regularizer=tf.keras.regularizers.l1(0.01))

# Una capa lineal con L2 regularization de factor 0.01 aplicado al bias del vector
keras.layers.Dense(64, bias_regularizer=tf.keras.regularizers.l2(0.01))

# Una capa lineal con el kernel inicializado a una matriz aleatoria ortogonal
keras.layers.Dense(64, kernel_initializer='orthogonal')

# Una capa lineal con un bias de 2.0
keras.layers.Dense(64, bias_initializer=tf.keras.initializers.constant(2.0))

La capa Dense implementa la operación: $output = activation(dot(input, kernel) + bias)$ donde activation es la función activación, kernel es la matriz de pesos de la capa y bias es el offset o bias del vector creado por la capa.  

Los argumentos posibles son:  

+ __units__: Cantidad de neuronas o dimensionalidad del espacio de salida.
+ __activation__: Funcion activación a usar. Si no se especifica, no se utiliza ninguna.
+ __use_bias__: Boolean. Si se utiliza o no un bias vector.
+ __kernel_initializer__: Inicializador de los pesos del kernel.
+ __bias_initializer__: Inicializador del vector de bias.
+ __kernel_regularizer__: Función regularización que se aplica al kernel. 
+ __bias_regularizer__: Función regularización que se aplica al vector de bias.
+ __activity_regularizer__: Función regularización que se aplica a la función activación.
+ __kernel_constraint__: Restricción que se aplica a los pesos.
+ __bias_constraint__: Restricción que se aplica al bias.  

***
### Entrenamiento y evaluación
#### Configuración de entrenamiento

Luego de que el modelo es contruido, se debe configurar el proceso de aprendizaje llamando al método compile:


In [5]:
model = tf.keras.Sequential([
# Agrego una capa fully-connected de 64 unidades
keras.layers.Dense(64, activation='relu'),
# Agrego otra igual
keras.layers.Dense(64, activation='relu'),
# Agrego una capa con activación softmax de 10 neuronas
keras.layers.Dense(10, activation='softmax')])

model.compile(optimizer=tf.train.AdamOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

tf.keras.Model.compile toma tres argumentos muy importantes: 
+ __optimizer__: Este objeto especifica el proceso de entrenamiento. https://www.tensorflow.org/api_docs/python/tf/train?hl=es
+ __loss__: La función a minimizar durante la optimización. https://www.tensorflow.org/api_docs/python/tf/keras/losses?hl=es
+ __metrics__: Métrica que se utiliza para monitorear el entrenamiento. https://www.tensorflow.org/api_docs/python/tf/keras/metrics?hl=es  

Adelante tenemos algunos ejemplos de configuración de entrenamiento:

In [6]:
# Configuración para regresión con error cuadrático medio 
model.compile(optimizer=tf.train.AdamOptimizer(0.01),
              loss='mse',       # mean squared error
              metrics=['mae'])  # mean absolute error

# Configuración para clasificación en categorías
model.compile(optimizer=tf.train.RMSPropOptimizer(0.01),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=[tf.keras.metrics.categorical_accuracy])

#### Datos de entrada tipo NumPy
Para pequeños datasets, se usa NumPy arrays para entrenar y evaluar los modelos. El modelo es entrenado por los datos de entrenamiento con el método fit:

In [7]:
data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))

model.fit(data, labels, epochs=10, batch_size=32)

Epoch 1/10
1000/1000 [==============================] - 0s 327us/step - loss: 11.6054 - categorical_accuracy: 0.1020
Epoch 2/10
1000/1000 [==============================] - 0s 49us/step - loss: 11.5629 - categorical_accuracy: 0.1150
Epoch 3/10
1000/1000 [==============================] - 0s 46us/step - loss: 11.5477 - categorical_accuracy: 0.1120
Epoch 4/10
1000/1000 [==============================] - 0s 44us/step - loss: 11.5445 - categorical_accuracy: 0.1100
Epoch 5/10
1000/1000 [==============================] - 0s 53us/step - loss: 11.5436 - categorical_accuracy: 0.1170
Epoch 6/10
1000/1000 [==============================] - 0s 50us/step - loss: 11.5413 - categorical_accuracy: 0.1070
Epoch 7/10
1000/1000 [==============================] - 0s 49us/step - loss: 11.5404 - categorical_accuracy: 0.1160
Epoch 8/10
1000/1000 [==============================] - 0s 43us/step - loss: 11.5398 - categorical_accuracy: 0.1100
Epoch 9/10
1000/1000 [==============================] - 0s 42us/step - 

tf.keras.Model.fit toma tres importantes argumentos:  
+ __epochs__: Una época es una iteración a lo largo de todo el set de entrenamiento (aunque se hace en lotes más pequeños).
+ __batch_size__: Cuando se le ingresa con NumPy data, el modelo divide los datos en lotes más pequeños (batchs) e itera sobre estos lotes durante el entrenamiento. Este número especifica el tamaño del lote. Hay que tener en cuenta que si el total de datos de entrada no es divisible por tamaño del lote, el último lote será más pequeño.
+ __validation_data__: cuando se evalua un modelo, se necesita monitorear su performance facilmente con datos de validación. Si se le pasa una tupla de entradas y otra de salidas esperadas o labels, el modelo puede mostrar la loss function y las métricas para los datos de validación al final de cada época. Es importante aclarar que estos datos no son usados para entrenar, es decir, no afectan a los pesos del kernel ni al bias.  

Aca tenemos un ejemplo utilizando datos de validación.

In [8]:
data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))

val_data = np.random.random((100, 32))
val_labels = np.random.random((100, 10))

model.fit(data, labels, epochs=10, batch_size=32,
          validation_data=(val_data, val_labels))

Train on 1000 samples, validate on 100 samples
Epoch 1/10
1000/1000 [==============================] - 0s 133us/step - loss: 11.4898 - categorical_accuracy: 0.1190 - val_loss: 11.6175 - val_categorical_accuracy: 0.1000
Epoch 2/10
1000/1000 [==============================] - 0s 48us/step - loss: 11.4868 - categorical_accuracy: 0.1190 - val_loss: 11.6115 - val_categorical_accuracy: 0.0900
Epoch 3/10
1000/1000 [==============================] - 0s 68us/step - loss: 11.4867 - categorical_accuracy: 0.0950 - val_loss: 11.6097 - val_categorical_accuracy: 0.1200
Epoch 4/10
1000/1000 [==============================] - 0s 66us/step - loss: 11.4853 - categorical_accuracy: 0.1280 - val_loss: 11.6269 - val_categorical_accuracy: 0.1100
Epoch 5/10
1000/1000 [==============================] - 0s 47us/step - loss: 11.4852 - categorical_accuracy: 0.1110 - val_loss: 11.6119 - val_categorical_accuracy: 0.1400
Epoch 6/10
1000/1000 [==============================] - 0s 46us/step - loss: 11.4824 - categorica

#### Importar datasets de tf.data
Para grandes datasets o entrenamiento en multiples dispositivos se utiliza la API Datasets. Se pasa un tf.data.Dataset al método fit.

In [9]:
# Instancias de un dataset
dataset = tf.data.Dataset.from_tensor_slices((data, labels))
dataset = dataset.batch(32)
dataset = dataset.repeat()

# No olvidat de especificar steps_per_epoch cuando se llama fit en un dataset
model.fit(dataset, epochs=10, steps_per_epoch=30)

Epoch 1/10
30/30 [==============================] - 0s 4ms/step - loss: 11.4581 - categorical_accuracy: 0.1333
Epoch 2/10
30/30 [==============================] - 0s 898us/step - loss: 11.5143 - categorical_accuracy: 0.1479
Epoch 3/10
30/30 [==============================] - 0s 898us/step - loss: 11.4787 - categorical_accuracy: 0.1479
Epoch 4/10
30/30 [==============================] - 0s 1ms/step - loss: 11.4774 - categorical_accuracy: 0.1604
Epoch 5/10
30/30 [==============================] - 0s 1ms/step - loss: 11.4685 - categorical_accuracy: 0.1760
Epoch 6/10
30/30 [==============================] - 0s 1ms/step - loss: 11.4817 - categorical_accuracy: 0.1677
Epoch 7/10
30/30 [==============================] - 0s 931us/step - loss: 11.4662 - categorical_accuracy: 0.1813
Epoch 8/10
30/30 [==============================] - 0s 997us/step - loss: 11.4698 - categorical_accuracy: 0.1813
Epoch 9/10
30/30 [==============================] - 0s 997us/step - loss: 11.4533 - categorical_accuracy

Acá, el método fit utiliza el argumento steps_per_epoch. Esto se refiere la cantidad de pasos de entrenamiento que el modelo corre antes de pasar a la siguiente época. COmo el Dataset ya viene en lotes de datos, no hace falta especificar el batch_size.  
Los datasets también se puede utilizar para validación.

In [10]:
dataset = tf.data.Dataset.from_tensor_slices((data, labels))
dataset = dataset.batch(32).repeat()

val_dataset = tf.data.Dataset.from_tensor_slices((val_data, val_labels))
val_dataset = val_dataset.batch(32).repeat()

model.fit(dataset, epochs=10, steps_per_epoch=30,
          validation_data=val_dataset,
          validation_steps=3)

Epoch 1/10
30/30 [==============================] - 0s 5ms/step - loss: 11.4155 - categorical_accuracy: 0.1521 - val_loss: 11.7025 - val_categorical_accuracy: 0.1042
Epoch 2/10
30/30 [==============================] - 0s 964us/step - loss: 11.4740 - categorical_accuracy: 0.1656 - val_loss: 11.2636 - val_categorical_accuracy: 0.0938
Epoch 3/10
30/30 [==============================] - 0s 1ms/step - loss: 11.4384 - categorical_accuracy: 0.1521 - val_loss: 11.3373 - val_categorical_accuracy: 0.0417
Epoch 4/10
30/30 [==============================] - 0s 1ms/step - loss: 11.4422 - categorical_accuracy: 0.1552 - val_loss: 11.5487 - val_categorical_accuracy: 0.0521
Epoch 5/10
30/30 [==============================] - 0s 1ms/step - loss: 11.4300 - categorical_accuracy: 0.1708 - val_loss: 11.6889 - val_categorical_accuracy: 0.0625
Epoch 6/10
30/30 [==============================] - 0s 1ms/step - loss: 11.4443 - categorical_accuracy: 0.1927 - val_loss: 11.2523 - val_categorical_accuracy: 0.0521
Ep

#### Evaluación y predicción
Los métodos tf.keras.Model.evaluate y tf.keras.Model.predict pueden usar NumPy data y tf.data.Dataset.  

Para evaluar el modo de inferencia de la loss function y de la métrica de los datos:

In [15]:
data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))

model.evaluate(data, labels, batch_size=32)

model.evaluate(dataset, steps=30)

30/30 [==============================] - 0s 2ms/step


[11.394348398844402, 0.18854166666666666]

Y para predecir la salida de la última capa según los datos de entrada (como NumPy array):

In [16]:
result = model.predict(data, batch_size=32)
print(result.shape)

(1000, 10)


***
### Construcción de modelo avanzados